# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [28]:
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [29]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [79]:
import pandas as pd
import os
from glob import glob

PRICE_DATA = os.getenv("PRICE_DATA")

parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
dd_px = dd.read_parquet(parquet_files).set_index("ticker")
dd_px

c:\Users\jonah\miniconda3\envs\dsi_participant\lib\site-packages\pandas\core\frame.py:717: DeprecationWarning: Passing a BlockManager to DataFrame is deprecated and will raise in a future version. Use public APIs instead.
  warnings.warn(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year
npartitions=90,,,,,,,,,
ACN,datetime64[ns],float64,float64,float64,float64,float64,float64,object,int32
AIMT,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...
ZIXI,...,...,...,...,...,...,...,...,...


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [80]:
# Write your code below.
dd_features = dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1 = x['Close'].shift(1),
        Adj_Close = x['Adj Close'].shift(1)))

dd_feat = dd_features.assign(
    returns = lambda x: x['Close']/x['Close_lag_1'] - 1,
    hi_lo_range = lambda x: x['High'] - x['Low'],)

dd_feat.head()

C:\Users\jonah\AppData\Local\Temp\ipykernel_14804\212168874.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_features = dd_px.groupby('ticker', group_keys=False).apply(


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close,returns,hi_lo_range
ticker,,,,,,,,,,,,,
ACN,2016-07-05,113.510002,113.559998,112.629997,113.099998,106.026108,2041100.0,ACN.csv,2016,NaN,NaN,NaN,0.930000
ACN,2016-07-06,112.629997,113.519997,111.959999,113.519997,106.419830,2667200.0,ACN.csv,2016,113.099998,106.026108,0.003714,1.559998
ACN,2016-07-07,113.510002,113.900002,112.510002,112.699997,105.651115,2050500.0,ACN.csv,2016,113.519997,106.419830,-0.007223,1.389999
ACN,2016-07-08,113.730003,115.239998,113.169998,115.110001,107.910400,1962800.0,ACN.csv,2016,112.699997,105.651115,0.021384,2.070000
ACN,2016-07-11,115.699997,115.980003,115.129997,115.459999,108.238510,1522300.0,ACN.csv,2016,115.110001,107.910400,0.003041,0.850006


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [81]:
# Write your code below.
pd_df = dd_feat.compute()
pd_df = pd_df.assign(
    rolling_mean = lambda x: x['returns'].rolling(10).mean())
pd_df


,Date,Open,High,Low,Close,Adj Close,Volume,source,Year,Close_lag_1,Adj_Close,returns,hi_lo_range,rolling_mean
ticker,,,,,,,,,,,,,,
ACN,2016-07-05,113.510002,113.559998,112.629997,113.099998,106.026108,2041100.0,ACN.csv,2016,NaN,NaN,NaN,0.930000,NaN
ACN,2016-07-06,112.629997,113.519997,111.959999,113.519997,106.419830,2667200.0,ACN.csv,2016,113.099998,106.026108,0.003714,1.559998,NaN
ACN,2016-07-07,113.510002,113.900002,112.510002,112.699997,105.651115,2050500.0,ACN.csv,2016,113.519997,106.419830,-0.007223,1.389999,NaN
ACN,2016-07-08,113.730003,115.239998,113.169998,115.110001,107.910400,1962800.0,ACN.csv,2016,112.699997,105.651115,0.021384,2.070000,NaN
ACN,2016-07-11,115.699997,115.980003,115.129997,115.459999,108.238510,1522300.0,ACN.csv,2016,115.110001,107.910400,0.003041,0.850006,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,2004-12-27,4.690000,4.970000,4.690000,4.880000,4.880000,544800.0,ZIXI.csv,2004,4.680000,4.680000,0.042735,0.280000,0.002948
ZIXI,2004-12-28,4.940000,5.380000,4.920000,5.190000,5.190000,1074200.0,ZIXI.csv,2004,4.880000,4.880000,0.063525,0.460000,0.009300
ZIXI,2004-12-29,5.150000,5.250000,5.100000,5.110000,5.110000,508100.0,ZIXI.csv,2004,5.190000,5.190000,-0.015414,0.150000,0.008390


Please comment:
+ Was it necessary to convert to pandas to calculate the moving average return?
Not necessary
+ Would it have been better to do it in Dask? Why?
As discussed in class, Dask is better suited for large datasets. In this case (small dataset), Pandas was sufficient.

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.